# [문제]
- law_2.docx 파일을 읽고, Chroma 저장
- LLM 질문 -> 질문
- 전세사기피해에 관한 법률 질문만 받기
- 이 외의 질문은 '답변을 할 수 없습니다.'

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

database = Chroma(
    collection_name='chroma-law2',
    persist_directory='./chroma2',
    embedding_function=embedding,
)

prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question : {query}
'''

keywords = ["전세사기", "보증금", "임대차", "피해자", "전세보증금", "보호", "경매", "확정일자"]

query = input("질문을 입력하세요 ▶ ")

if not any(keyword in query for keyword in keywords):
    print("죄송합니다. 답변을 드릴 수 없습니다.")
else:
    retrieved_docs = database.similarity_search(query=query, k=3)

    formatted_prompt = prompt.format(
        retrieved_docs=retrieved_docs,
        query=query
    )

    llm = ChatOpenAI(model='gpt-4o')
    ai_message = llm.invoke(formatted_prompt)
    print(ai_message.content)